In [23]:
import sys 
import os
lib_path = './lib'
sys.path.insert(0, lib_path)     ## add lib path to sys path 
import pandas as pd
import MultiContagion as mc
import igraph
import random
import numpy as np
from matplotlib import pylab as plt
import scipy.stats
import copy
#import powerlaw
%matplotlib inline

First define some local folder path

In [54]:
## First define some local folder path
data_path = './data/0_CPIS_CDIS_BIS_USTIC_merged_fixed4_12102017.dta'
inflation_data_path = "./data/USA-PCPIE.csv"
csv_files_path = './data/csv_files'
export_path_csv = './results/csv_structuralmeasurements/'
adj_matrix_path = './data/adjacency_matrix/'

In [9]:
## Read data from stata file 
df = pd.read_stata(data_path )
df = df.sort_values(by=['country'])
#by argument to sort_index is deprecated, pls use .sort_values(by=...)
df = df.fillna(value = 0.)
df.head()

,countrycode,counterpart_code,country,counterpart,year,CDIS_IADE,CDIS_IADD,CDIS_IAD,CPIS_IAP,CPIS_IAPE,CPIS_IAPD,loans_dep,total_claims
64495,512.0,112.0,"Afghanistan, Islamic Republic of",United Kingdom,2013.0,0.000000,0.0,0.000000,0.0,0.0,0.0,964.0,964.000000
64649,512.0,542.0,"Afghanistan, Islamic Republic of","Korea, Republic of",2009.0,4.050179,0.0,4.050179,0.0,0.0,0.0,13.0,17.050179
64650,512.0,542.0,"Afghanistan, Islamic Republic of","Korea, Republic of",2010.0,4.956533,0.0,4.956533,0.0,0.0,0.0,5.0,9.956533
64651,512.0,542.0,"Afghanistan, Islamic Republic of","Korea, Republic of",2011.0,4.683440,0.0,4.683440,0.0,0.0,0.0,5.5,10.183439
64652,512.0,542.0,"Afghanistan, Islamic Republic of","Korea, Republic of",2012.0,5.475813,0.0,5.475813,0.0,0.0,0.0,6.0,11.475813


In [10]:
df_inflation = pd.read_csv(inflation_data_path, index_col="Year")
-df_inflation["Inflation"][2009] + df_inflation["Inflation"][2015]

20.802333333300027

In [11]:
#df_inflation = df_inflation.multiply(1000)

In [12]:
df_inflation["Inflation"][2015]

238.00433333330002

**Note on unities, inflation is in billion dollars and surveys are in million dollars **

[1] "Country Name"                                                                    
 [2] "Country Code"                                                                    
 [3] "Counterpart Country Name"                                                        
 [4] "Counterpart Country Code"                                                        
 [5] "Time Period"                                                                     
 [6] "IIP Equity investment asset, millions of USD"                                    
 [7] "IIP Debt instrument asset, millions of USD"                                      
 [8] "IIP Direct investment asset, millions of USD"                                    
 [9] "IIP Direct investment asset between fellow enterprise, millions of USD"          
[10] "IIP Total Portfolio invertment, Assets, millions of USD"                         
[11] "IIP Portfolio invertment,Equity and investment fund shares, Assets, millions of "

[12] "IIP Portfolio invertment,Debt securities, Assets, millions of USD"               
[13] "BIS loans and deposit claims positions, USD" <\br>

layers should be: 6, 7, 11, 12 and 13

In [7]:
df;

# CDIS Equity Layer

## The contagion network

In [13]:
def make_cont_net_from_survey_year_no_isolated(df, data, year):
    country = df["country"][ (df[data].notnull()) & (df[data] > 0 ) & (df["year"] == year )]
    countercountry = df["counterpart"][ (df[data].notnull()) & (df[data] > 0 ) & (df["year"] == year)  ]
    survey = df[data][ (df[data].notnull()) & (df[data] > 0 ) & (df["year"] == year) ]
    edges, weight = mc.make_edge_list(countercountry,country, survey)
    G = mc.make_graph_from_edge(edges, weight)
    return G

In [14]:
Year = 2016

In [15]:
def make_graphs(y):
    """graphs with no isolated countries, but then dimensions between matrices dont match"""
    G_cd_equity_15 = make_cont_net_from_survey_year_no_isolated(df, "CDIS_IADE", y)
    G_cd_debt_15 = make_cont_net_from_survey_year_no_isolated(df, "CDIS_IADD", y)
    G_cp_equity_15 = make_cont_net_from_survey_year_no_isolated(df, "CPIS_IAPE", y)
    G_cp_debt_15 = make_cont_net_from_survey_year_no_isolated(df, "CPIS_IAPD", y)
    G_bis_15 = make_cont_net_from_survey_year_no_isolated(df, "loans_dep", y)
    G_15_list = [G_cd_equity_15, G_cd_debt_15, G_cp_equity_15, G_cp_debt_15, G_bis_15  ]
    G_names = ["CDIS_equity", "CDIS_debt", "CPIS_equity", "CPIS_debt", "BIS" ]
    G_adjacency_15 = []
    
    for g in G_15_list:
        G_adjacency_15.append(np.array((g.get_adjacency(attribute= "weight")).data))
    
    return G_15_list, G_names, G_adjacency_15#, G_agg_2015

In [16]:
def make_agg_graph_local(y):
    G_cd_equity_15 = mc.make_cont_net_from_survey_year(df, "CDIS_IADE", y)
    G_cd_debt_15 = mc.make_cont_net_from_survey_year(df, "CDIS_IADD", y)
    G_cp_equity_15 = mc.make_cont_net_from_survey_year(df, "CPIS_IAPE",y)
    G_cp_debt_15 = mc.make_cont_net_from_survey_year(df, "CPIS_IAPD", y)
    G_bis_15 = mc.make_cont_net_from_survey_year(df, "loans_dep", y)
    G_15_list = [G_cd_equity_15, G_cd_debt_15, G_cp_equity_15, G_cp_debt_15, G_bis_15  ]
    G_adjacency_15 = []
    for g in G_15_list:
        G_adjacency_15.append(np.array((g.get_adjacency(attribute= "weight")).data))

    AM_agg_2015 = G_adjacency_15[0] + G_adjacency_15[1] + G_adjacency_15[2] + G_adjacency_15[3] + G_adjacency_15[4]
    G_agg_2015 = igraph.Graph.Weighted_Adjacency(AM_agg_2015.tolist(), attr= "weight")
    
    return G_agg_2015, AM_agg_2015

In [17]:
G_list, G_nm, G_adj = make_graphs(Year)
G_agg, G_adj_agg = make_agg_graph_local(Year)

In [18]:
len(G_adj[2][0]) # we excluded some nodes
len( G_adj_agg), len(G_adj_agg[0])

(212, 212)

In [19]:
dict_survey_names = {0: "CDIS-equity", 1: "CDIS-debt",2: "CPIS-equity",3: "CPIS-debt",4: "BIS"}

In [31]:
name = "AM4_no_isolated"
for y in range(2008,2017):
    G_list, G_nm, G_adj = make_graphs(y)
    for i in range(5):
        survey_name = dict_survey_names[i]
        np.savetxt(os.path.join(csv_files_path,name+survey_name+str(y)+".csv"), G_adj[i], delimiter=",")

In [41]:
## double check, see if save and load yield the same matrix 
my_data2 = np.genfromtxt(os.path.join(csv_files_path,'AM4_no_isolatedCDIS-equity2016.csv'), delimiter=',')
G_adj[0] == my_data2

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

### Save basic measurments

In [45]:
#export_path_csv = "csv_structuralmeasurements/"

def save_basic_measurements(y, list_of_graphs, names_of_graphs, exp_path=export_path_csv):
    '''Saves into a csv file the basic structural measurements of list of graphs
    Args:
        y(int): integer used to name the file
        list_of_graphs(list): list contaning graph type elements
        names_of_graphs(list): list containing the names of the graphs
    '''
    f = open(exp_path + "monoplex_basic_struc_measures"+ str(y) +".csv"\
            , "w")
    f.write("network,diameter (unweighted-directed),density(unweighted-directed),\
            mean geodesic distance(unweighted-directed), \
            reciprocity(unweighted-directed),\
            global clustering(unweighted-undirected),\
            average clustering (unweighted-undirected),\
            sum of weights\n")
    if len(list_of_graphs) != len(names_of_graphs):
        print ("warning dimention of lists do not match")
    n = len(list_of_graphs)
    for i in range(n):
        f.write(names_of_graphs[i] + ",")
        g = list_of_graphs[i]
        weights = np.array(copy.deepcopy(g.es["weight"]))
        f.write(str(round(g.diameter(), 2) ) + ",")
        f.write(str(round(g.density(), 2) ) + ",")
        f.write(str(round(g.average_path_length(), 2) ) + ",")
        f.write(str(round(g.reciprocity(), 2) ) + ",")
        f.write(str(round(g.transitivity_undirected(), 2) ) + ",")
        f.write(str(round(np.mean(g.transitivity_local_undirected(mode="zero"))\
                , 2) ) + ",")
        f.write(str(round(sum(weights)) ) + "\n")
    f.close()

In [46]:
#export_path_csv = "csv_structuralmeasurements/"
for Year in [2008 + i for i in range(9)]:
    print(Year)
    G_list, G_nm, G_adj = make_graphs(Year)
    G_agg = make_agg_graph_local(Year)[0]
    G_list_plus_agg = G_list + [G_agg]
    G_nm_plus_agg = G_nm + ["Aggregated"]
    save_basic_measurements(Year, G_list_plus_agg, G_nm_plus_agg)

2008
2009
2010
2011
2012
2013
2014
2015
2016


### Distance between networks of different years

### we need matrices with all the nodes, even the ones that were not reported in that year

In [47]:
Year = 2016

In [48]:
df = pd.read_stata(data_path)
df = df.sort_values(by=['country'])
df = df.fillna(value = 0.)

In [49]:
df;

In [50]:
def make_cont_net_from_survey_year_all_countries(df, data, year):
    all_years = [i for i in range(2008,2017)]
    all_years.remove(year)
    #print all_years
    country = df["country"]
    countercountry = df["counterpart"]
    DF = df.copy()
    DF.ix[df.year.isin(all_years), data] = 0
    survey = DF[data]
    edges, weight = mc.make_edge_list(countercountry,country, survey)
    for i in range(len(weight)):
        if weight[i] < 0:
            weight[i] = 0
    G = mc.make_graph_from_edge(edges, weight)
   
    return G


In [51]:
def make_graphs_allnodes(y, norm = False):
    '''Dimensions of matrices match even between years'''
    df_copy = copy.deepcopy(df)
    if norm == True:
        df_copy[['CDIS_IADE', 'CDIS_IADE', 'CPIS_IAPE', 'CPIS_IAPD', 'loans_dep' ] ] *= 1./df_inflation["Inflation"][y]
    G_cd_equity_15 = make_cont_net_from_survey_year_all_countries(df_copy, "CDIS_IADE", y)
    G_cd_debt_15 = make_cont_net_from_survey_year_all_countries(df_copy, "CDIS_IADD", y)
    G_cp_equity_15 = make_cont_net_from_survey_year_all_countries(df_copy, "CPIS_IAPE", y)
    G_cp_debt_15 = make_cont_net_from_survey_year_all_countries(df_copy, "CPIS_IAPD", y)
    G_bis_15 = make_cont_net_from_survey_year_all_countries(df_copy, "loans_dep", y)
    G_15_list = [G_cd_equity_15, G_cd_debt_15, G_cp_equity_15, G_cp_debt_15, G_bis_15  ]
    G_names = ["CDIS_equity", "CDIS_debt", "CPIS_equity", "CPIS_debt", "BIS" ]
    G_adjacency_15 = []
    
    for g in G_15_list:
        G_adjacency_15.append(np.array((g.get_adjacency(attribute= "weight")).data))
    #print G_15_list[0].vs["name"][0]
    return G_15_list, G_names, G_adjacency_15#, G_agg_2015

In [52]:
def make_agg_graph(y, norm = False):
    df_copy = copy.deepcopy(df)
    if norm == True:
        df_copy[['CDIS_IADE', 'CDIS_IADE', 'CPIS_IAPE', 'CPIS_IAPD', 'loans_dep' ] ] *= 1./df_inflation["Inflation"][y]
    G_cd_equity_15 = make_cont_net_from_survey_year_all_countries(df_copy, "CDIS_IADE", y)
    G_cd_debt_15 = make_cont_net_from_survey_year_all_countries(df_copy, "CDIS_IADD", y)
    G_cp_equity_15 = make_cont_net_from_survey_year_all_countries(df_copy, "CPIS_IAPE",y)
    G_cp_debt_15 = make_cont_net_from_survey_year_all_countries(df_copy, "CPIS_IAPD", y)
    G_bis_15 = make_cont_net_from_survey_year_all_countries(df_copy, "loans_dep", y)
    G_15_list = [G_cd_equity_15, G_cd_debt_15, G_cp_equity_15, G_cp_debt_15, G_bis_15  ]
    G_adjacency_15 = []
    for g in G_15_list:
        G_adjacency_15.append(np.array((g.get_adjacency(attribute= "weight")).data))

    AM_agg_2015 = G_adjacency_15[0] + G_adjacency_15[1] + G_adjacency_15[2] + G_adjacency_15[3] + G_adjacency_15[4]
    G_agg_2015 = igraph.Graph.Weighted_Adjacency(AM_agg_2015.tolist(), attr= "weight")
    
    return G_agg_2015, AM_agg_2015

In [53]:
dict_survey_names = {0: "CDIS-equity", 1: "CDIS-debt",2: "CPIS-equity",3: "CPIS-debt",4: "BIS"}

In [56]:
name = "AM4_all_nodes"
for y in range(2008,2017):
    G_list, G_nm, G_adj = make_graphs_allnodes(y, norm = False)
    for i in range(5):
        survey_name = dict_survey_names[i]
        np.savetxt(adj_matrix_path+name+survey_name+str(y)+".csv", G_adj[i], delimiter=",")

D:\data\chengyu\_GFF\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [57]:
name = "AM4_all_nodes_aggregate"
for y in range(2008,2017):
    Ga_list, G_adj = make_agg_graph(y, norm = False)
    np.savetxt(adj_matrix_path+name+str(y)+".csv", G_adj, delimiter=",")

D:\data\chengyu\_GFF\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [58]:
all_names = G_list[0].vs["name"]

In [59]:
len( G_list[3].vs["name"])

213

In [60]:
len(all_names)

213

In [61]:
all_names = G_list[0].vs["name"]
all_names = [n.replace(',',' ') for n in all_names]
f = open(adj_matrix_path+"all_country_name4"+".csv", "w")
for i in all_names:
    f.write(i+"\n")
f.close()